In [0]:
%sql
create catalog sesion_11

In [0]:
%sql
create schema sesion_11.auditoria

In [0]:
%sql
create schema sesion_11.gold

In [0]:
%sql
CREATE TABLE sesion_11.auditoria.ingestion_log (
  job_id BIGINT NOT NULL COMMENT 'Job id del workflow',
  job_run_id BIGINT NOT NULL COMMENT 'Job run id del workflow',
  task_run_id BIGINT NOT NULL COMMENT 'Task run id del workflow',
  job_start_time TIMESTAMP NOT NULL COMMENT 'Fecha inicio del workflow',
  job_end_time TIMESTAMP NOT NULL COMMENT 'Fecha fin del workflow',
  job_duration_seconds BIGINT NOT NULL COMMENT 'Duración del workflow',
  job_status STRING NOT NULL COMMENT 'Estado del workflow',
  table STRING NOT NULL COMMENT 'Nombre de la tabla',
  layer STRING NOT NULL COMMENT 'Capa de la tabla',
  rows_in BIGINT NOT NULL COMMENT 'Registros ingresados',
  rows_inserted BIGINT NOT NULL COMMENT 'Registros insertados',
  rows_updated BIGINT NOT NULL COMMENT 'Registros actualizados',
  rows_deleted BIGINT NOT NULL COMMENT 'Registros eliminados',
  file_bytes BIGINT NOT NULL COMMENT 'Peso del archivo delta',
  merge_duration_seconds BIGINT NOT NULL COMMENT 'Duración de la operación merge',
  creation_date TIMESTAMP NOT NULL
    DEFAULT CURRENT_TIMESTAMP()
    COMMENT 'Fecha de creación de la operación'
) USING delta
COMMENT 'Tabla que contiene la trazabilidad de una ingesta de datos.'
TBLPROPERTIES (
  'delta.autoOptimize.autoCompact' = 'True',
  'delta.autoOptimize.optimizeWrite' = 'True',
  'delta.feature.allowColumnDefaults' = 'supported',
  'vorder.enabled' = 'True'
)

In [0]:
df_ventas = spark.read.csv("/Volumes/sesion_11/default/input/ventas_base.csv")

In [0]:
#df_ventas.write.format("delta").mode("append").saveAsTable("sesion_11.gold.ventas")
df_ventas.write.format("delta").mode("overwrite").options("header","true").saveAsTable("sesion_11.gold.ventas")

In [0]:
%sql
describe history sesion_11.gold.ventas

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import lit

delta_table = DeltaTable.forName(spark, "sesion_11.gold.ventas")

merge = (
    delta_table.alias("t")
    .merge(
        df_ventas.alias("s"),
        lit(False),
    )
    .whenNotMatchedInsertAll()
    .execute()
)